# Example: Lattice Model of Interest Rates
Fill me in

## Setup
Set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, various functions we will use in the exercise, and custom types to model the components of our example problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl). 

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2024/lectures/week-3/L3a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-CourseRepository-Fall-2

## Prerequisite: Load the Treasury Rates Dataset
Fill me in

In [5]:
tbill_rates_dataset = CSV.read(joinpath(_PATH_TO_DATA, "UST-Daily-TBill-Rates-2024.csv"), DataFrame) |> reverse

Row,Date,4 WEEKS BANK DISCOUNT,4 WEEKS COUPON EQUIVALENT,8 WEEKS BANK DISCOUNT,8 WEEKS COUPON EQUIVALENT,13 WEEKS BANK DISCOUNT,13 WEEKS COUPON EQUIVALENT,17 WEEKS BANK DISCOUNT,17 WEEKS COUPON EQUIVALENT,26 WEEKS BANK DISCOUNT,26 WEEKS COUPON EQUIVALENT,52 WEEKS BANK DISCOUNT,52 WEEKS COUPON EQUIVALENT
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,01/02/2024,5.28,5.39,5.28,5.41,5.22,5.38,5.21,5.39,5.04,5.26,4.57,4.81
2,01/03/2024,5.27,5.38,5.28,5.41,5.24,5.4,5.21,5.39,5.06,5.28,4.58,4.82
3,01/04/2024,5.29,5.4,5.28,5.41,5.24,5.4,5.21,5.39,5.06,5.28,4.61,4.85
4,01/05/2024,5.28,5.39,5.27,5.4,5.24,5.4,5.2,5.38,5.05,5.27,4.61,4.85
5,01/08/2024,5.28,5.39,5.28,5.41,5.25,5.41,5.19,5.37,5.04,5.26,4.59,4.83
6,01/09/2024,5.28,5.39,5.26,5.39,5.24,5.4,5.19,5.37,5.04,5.26,4.6,4.84
7,01/10/2024,5.27,5.38,5.26,5.39,5.23,5.39,5.19,5.37,5.04,5.26,4.6,4.84
8,01/11/2024,5.28,5.39,5.27,5.4,5.23,5.39,5.18,5.36,5.03,5.25,4.54,4.77
9,01/12/2024,5.28,5.39,5.27,5.4,5.22,5.37,5.17,5.35,4.98,5.19,4.45,4.67


Fill me in

In [7]:
Δt = (1/252); 

## Task 1: Build a Lattice Model and look at its properties
Fill me in

In [9]:
p = 0.50;
u = 1.25;
d = 1/u;
T = 2; # this will give T + 1 levels (zero based)
rₒ = 0.1;

Fill me in

In [11]:
my_lattice_tree_model = build(MySymmetricBinaryInterestRateLatticeModel, (
    p = p, u = u, d = d, T = T, rₒ = rₒ
)) |> populate;

Fill me in

In [13]:
my_lattice_tree_model.data

Dict{Int64, MyBinaryInterestRateLatticeNodeModel} with 6 entries:
  0 => MyBinaryInterestRateLatticeNodeModel(1.0, 0.1, 8.0e-323)
  4 => MyBinaryInterestRateLatticeNodeModel(0.5, 0.1, 2.20585e-314)
  5 => MyBinaryInterestRateLatticeNodeModel(0.25, 0.064, 2.0e-323)
  2 => MyBinaryInterestRateLatticeNodeModel(0.5, 0.08, 2.10297e-305)
  3 => MyBinaryInterestRateLatticeNodeModel(0.25, 0.15625, 4.95825e-319)
  1 => MyBinaryInterestRateLatticeNodeModel(0.5, 0.125, 2.12654e-314)

Fill me in

In [15]:
my_lattice_tree_model.connectivity

Dict{Int64, Vector{Int64}} with 3 entries:
  0 => [1, 2]
  2 => [4, 5]
  1 => [3, 4]

Fill me in

In [17]:
my_lattice_tree_model.levels

Dict{Int64, Vector{Int64}} with 3 entries:
  0 => [0]
  2 => [3, 4, 5]
  1 => [1, 2]

## Task 2: Estimate the Lattice Model Parameters from Data
Fill me in

In [19]:
key_column_name = "52 WEEKS BANK DISCOUNT"; # you can select any col, but let's pick the 1-year T-bill

Fill me in

In [82]:
number_of_values = tbill_rates_dataset[:,key_column_name] |> length;
rates_return_array = Array{Float64,1}(undef, number_of_values - 1);
for i ∈ 2:number_of_values
    rᵢ = tbill_rates_dataset[i-1,key_column_name] |> x-> x/100.0;
    r₂ = tbill_rates_dataset[i,key_column_name] |> x-> x/100.0;
    rates_return_array[i-1] = (1/Δt)*log(r₂/rᵢ)
end

### Visualize rate returns
Fill me in

In [86]:
## fill me in

Up factor

In [74]:
i = findall(x-> x > 0, rates_return_array);
u = rates_return_array[i] |> mean |> x-> exp(x*Δt); # increase per day

Down factor

In [76]:
j = findall(x-> x < 0, rates_return_array);
d = rates_return_array[j] |> mean |> x-> exp(x*Δt); # average decrease per day

Probability

In [79]:
p = length(i)/(number_of_values - 1)

0.42483660130718953

### TODO: Simulate the short and forward rates
Fill me in

In [89]:
### fill me in

## Task 3: Zero Price Simulation
Fill me in

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.